In [1]:
# Run only on the first execution
!python setup.py build develop

running build
running build_py
creating build/lib.linux-x86_64-3.10
creating build/lib.linux-x86_64-3.10/tools
copying tools/lazyconfig_train_net.py -> build/lib.linux-x86_64-3.10/tools
copying tools/train_net.py -> build/lib.linux-x86_64-3.10/tools
copying tools/__init__.py -> build/lib.linux-x86_64-3.10/tools
copying tools/benchmark.py -> build/lib.linux-x86_64-3.10/tools
copying tools/lightning_train_net.py -> build/lib.linux-x86_64-3.10/tools
copying tools/visualize_json_results.py -> build/lib.linux-x86_64-3.10/tools
copying tools/visualize_data.py -> build/lib.linux-x86_64-3.10/tools
copying tools/convert-torchvision-to-d2.py -> build/lib.linux-x86_64-3.10/tools
copying tools/analyze_model.py -> build/lib.linux-x86_64-3.10/tools
copying tools/plain_train_net.py -> build/lib.linux-x86_64-3.10/tools
creating build/lib.linux-x86_64-3.10/detectron2
copying detectron2/__init__.py -> build/lib.linux-x86_64-3.10/detectron2
creating build/lib.linux-x86_64-3.10/detectron2/structures
copyi

In [2]:
import os
import copy
import torch
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader, build_detection_train_loader

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Register Dataset
try:
    register_coco_instances('coco_trash_train', {}, '../../../dataset/train.json', '../../../dataset/')
except AssertionError:
    pass

try:
    register_coco_instances('coco_trash_test', {}, '../../../dataset/test.json', '../../../dataset/')
except AssertionError:
    pass

MetadataCatalog.get('coco_trash_train').thing_classes = ["General trash", "Paper", "Paper pack", "Metal", 
                                                         "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

In [4]:
# config 불러오기
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))

In [5]:
# config 수정하기
cfg.DATASETS.TRAIN = ('coco_trash_train',)
cfg.DATASETS.TEST = ('coco_trash_test',)

cfg.DATALOADER.NUM_WOREKRS = 0

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml')

cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 15000
cfg.SOLVER.STEPS = (8000,12000)
cfg.SOLVER.GAMMA = 0.005
cfg.SOLVER.CHECKPOINT_PERIOD = 3000

cfg.OUTPUT_DIR = './output'

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 12

cfg.TEST.EVAL_PERIOD = 3000

In [6]:
# mapper - input data를 어떤 형식으로 return할지 (따라서 augmnentation 등 데이터 전처리 포함 됨)
import detectron2.data.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
def MyMapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    transform_list = [
        # T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        # T.RandomBrightness(0.8, 1.8),
        # T.RandomContrast(0.6, 1.3)
        A.Resize(1024, 1024),
        ToTensorV2(p=0.1)

    ]
    
    image, transforms = T.apply_transform_gens(transform_list, image)
    
    dataset_dict['image'] = torch.as_tensor(image.transpose(2,0,1).astype('float32'))
    
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop('annotations')
        if obj.get('iscrowd', 0) == 0
    ]
    
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict['instances'] = utils.filter_empty_instances(instances)
    
    return dataset_dict

In [7]:
# trainer - DefaultTrainer를 상속
class MyTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
        cfg, mapper = MyMapper, sampler = sampler
        )
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs('./output_eval', exist_ok = True)
            output_folder = './output_eval'
            
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
# train
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()